In [1]:
#import nltk
#nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ezequiel\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [1]:
#Import.
import gzip
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import unicodedata
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import one_hot
from ast import literal_eval

def jl_to_list(fname):
    output = []
    with gzip.open(fname, 'rb') as f:
        for line in f:
            output.append(json.loads(line))
    return output

def load_item_data(all_itms = False):
    ITEM_DATA = pd.read_csv('item_data.csv', sep=';')
    fields = ['item_id', 'domain_id', 'product_id', 'category_id', 'price', 'price_cluster', 'condition', 'mexico']
    m = {}
    for column in tqdm(fields):
         m[column] = list(ITEM_DATA[column])
    metadata = {}
    for i, j in tqdm(enumerate(m['item_id'])):
        metadata[j] = {}
        for column in fields: 
            metadata[j].update({column: m[column][i]})
    if all_itms:
        all_items = list(metadata)
    else:
        all_items = []
    return metadata, all_items

def unicode_to_ascii(s):
    return unicodedata.normalize('NFKD', s).encode('ASCII', 'ignore')

def searchs(row):
    return ([ev['event_info'] for ev in row['user_history'] if ev['event_type']=='search'])

def process_unicode_str(s):
    s = unicode_to_ascii(s).decode("utf-8") 
    return s.replace(',', ' ').strip().upper()

process_unicode_str_v = np.vectorize(process_unicode_str)

def token_palabra(s, n, l):
    if n < l:
        return word_tokenize(s)[n]
    else:
        return ''

token_palabra_v = np.vectorize(token_palabra, excluded=['n'])

def len_title(s):
    return len(word_tokenize(s))

len_title_v = np.vectorize(len_title)

def split_list(L):
    words = []
    if len(L) > 0 :
        for l in L:
            for s in l.split():
                words.append(s)
    words = set(words)
    s = ''
    for w in words:
        s += w + ' '
    return s[:-1]

def words_to_int(s, wl):
    return one_hot(s, wl)

words_to_int_v = np.vectorize(words_to_int)

def words_to_int_l(S, wl):
    return [words_to_int(s, wl) for s in S]

def words_to_int2(s, wl):
    return str(one_hot(s, wl))

words_to_int2_v = np.vectorize(words_to_int2)

def ranking_by_search(s,P):
    dicc = {}
    for w in s:
        if w in P:
            dicc = {key: dicc.get(key, 0) + P[w].get(key, 0) for key in set(dicc) | set(P[w])}
    return dicc

#Se requiere una versión que restrinja por mercado.
#Y otra que limpie la endogeneidad. Sino la va a sobrevaluar.
def join_rankings(S, P):
    dicc = {}
    for i, s in enumerate(S):
        d = ranking_by_search(s, P)
        dicc = {key: dicc.get(key, 0) + d.get(key, 0) for key in set(dicc) | set(d)}
    return [x[0] for x in sorted(dicc.items(), key=lambda item: item[1], reverse=True)][:10]

def join_rankings_s(S, P):
    dicc = {}
    for i, s in enumerate(S):
        d = ranking_by_search(s, P)
        dicc = {key: dicc.get(key, 0) + d.get(key, 0) for key in set(dicc) | set(d)}
    return str([x[0] for x in sorted(dicc.items(), key=lambda item: item[1], reverse=True)][:10])

join_rankings_v = np.vectorize(join_rankings_s, excluded=['P'])

In [2]:
ITEM_DATA = pd.read_csv('item_data.csv', sep=';')

In [3]:
title = np.array(ITEM_DATA['title'])

In [4]:
%%time
ninja = process_unicode_str_v(title)

Wall time: 3.32 s


# Cuenta de palabras.

In [15]:
%%time
linyera = len_title_v(ninja)

Wall time: 2min 31s


In [7]:
crayon = np.empty([title.size, linyera.max()], dtype=object)

In [8]:
for i in tqdm(range(crayon.shape[1])):
    crayon[:,i] = token_palabra_v(ninja, i, linyera)

In [15]:
CRAYON = pd.DataFrame(crayon, columns=['word'+str(i) for i in range(crayon.shape[1])])

In [17]:
#CRAYON.to_csv('matrix_of_words_titles.csv', sep=';', index=False)

In [42]:
#CRAYON = pd.read_csv('matrix_of_words_titles.csv', sep=';')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Carga de datos de búsquedas.

In [5]:
ss_train = pd.read_csv('searchs_train_uq.csv', sep=';')
ss_test = pd.read_csv('searchs_test_uq.csv', sep=';')

In [6]:
ss_train['searchs_train_uq'] = ss_train['searchs_train_uq'].apply(lambda x: literal_eval(x))
ss_test['searchs_test_uq'] = ss_test['searchs_test_uq'].apply(lambda x: literal_eval(x))

In [7]:
ss_train['words'] = ss_train['searchs_train_uq'].apply(split_list)
ss_test['words'] = ss_test['searchs_test_uq'].apply(split_list)

# Retorna a cuenta de palabras.

In [47]:
s_train = np.array(ss_train['words'])
s_test = np.array(ss_test['words'])

In [48]:
words_train = []
for r in tqdm(range(len(ss_train))):
    words_train.extend(ss_train['words'].iloc[r].split())
    
words_train = set(words_train)

In [49]:
words_test = []
for r in tqdm(range(len(ss_test))):
    words_test.extend(ss_test['words'].iloc[r].split())
    
words_test = set(words_test)

In [50]:
len(words_train)

121002

In [51]:
len(words_test)

77049

In [52]:
inter = words_train.intersection(words_test)

In [53]:
len(inter)

51474

In [24]:
words = words_train.union(words_test)

In [17]:
for i in tqdm(range(crayon.shape[1])):
    if i == 0:
        words_title = set(np.unique(crayon[:,i]))
    else:
        words_title = words_title.union(set(np.unique(crayon[:,i])))

In [16]:
#words_title = set(np.unique(crayon))

MemoryError: 

In [25]:
#La primera vez que hice este conjunto estaba mal. Originalmente len(words)==548326. Ahora resultan 562373.
#words = words_train.union(words_title)
words = words.union(words_title)

In [26]:
len(words)

562373

In [8]:
w_len = len(words) + 500 - 326
w_len = len(words) + 600 - 373

# Retorno a confección de datos.

In [9]:
busquedas = pd.concat([ss_train['searchs_train_uq'], ss_test['searchs_test_uq']], axis=0,)

In [10]:
busquedas = busquedas.reset_index()

In [11]:
busquedas.columns = ['index', 'searchs']

In [12]:
busquedas

,index,searchs
0,0,[RELOGIO SMARTWATCH]
1,1,[DESMAMADEIRA ELETRICA]
2,2,[]
3,3,"[RADIOBOSS, SOUND FORGE, SOUND FORGE PLUGINS]"
4,4,"[AMAZFIT BIP, AMAZFIT BIPAMAZFIT BIP, AMAZFIT ..."
...,...,...
590228,177065,"[ENDER 3, MOTOR TUBULAR PERSIANA]"
590229,177066,"[CAJA PORTAOBJETOS LABORATORIO, FILTRO AGUA, F..."
590230,177067,"[CONTROLE CELULAR, FANTASIAS HALLOWEEN CORINGA]"
590231,177068,[]


In [14]:
#Con este esquema llegué a 51256 palabras coincidentes.
#La segunda vez fueron 51162 palabras coincidentes.
w_len = 562373 + 600 - 373
busquedas['searchs_int'] = busquedas['searchs'].apply(words_to_int_l, args=[w_len])

In [65]:
#ss_test['searchs_int'] = ss_test['searchs_test_uq'].apply(words_to_int_l, args=[w_len])

In [10]:
#ss_train['searchs_int'] = ss_train['searchs_train_uq'].apply(words_to_int_l, args=[w_len])

In [15]:
words_int_train = []
for S in list(busquedas['searchs_int'][:413163]):
    for s in S:
        for w in s:
            words_int_train.append(w)

In [16]:
words_int_test = []
for S in list(busquedas['searchs_int'][-177070:]):
    for s in S:
        for w in s:
            words_int_test.append(w)

In [17]:
words_int_train = set(words_int_train)

In [18]:
words_int_test = set(words_int_test)

In [19]:
len(words_int_test.intersection(words_int_train))

51162

In [20]:
ss_train = pd.concat([ss_train, busquedas['searchs_int'][:413163]], axis=1)

In [21]:
temp = busquedas['searchs_int'][-177070:]

In [22]:
temp = temp.reset_index()

In [23]:
ss_test = pd.concat([ss_test, temp['searchs_int']], axis=1)

In [24]:
IB = pd.read_csv('ib.csv', sep=';')

In [25]:
ss_train = pd.concat([ss_train, IB], axis=1)

# Retorno a títulos de ítems.

In [26]:
#Controlar que las palabras aquí estén puestas con criterio. Sino 36 y 12 pueden estar mal asignados y habría que cambiar la estrategia.
turtle = words_to_int2_v(ninja, w_len)

In [31]:
word_int_titles = []
for i in tqdm(list(turtle)):
    for j in literal_eval(i):
        word_int_titles.append(j)

In [32]:
word_int_titles = set(word_int_titles)
len(word_int_titles)

270621

In [34]:
print(len(word_int_titles.intersection(words_int_train)))
print(len(word_int_titles.intersection(words_int_test)))

89687


In [38]:
TURTLE = pd.DataFrame(turtle, columns=['encoded_title'])

In [39]:
ITEM_DATA = pd.concat([ITEM_DATA, TURTLE], axis=1)

In [40]:
ss_train = pd.merge(ss_train, 
                 ITEM_DATA.loc[:,['item_id', 'encoded_title', 'title']], 
                 how='left', 
                 left_on='ib', 
                 right_on='item_id').drop(columns=['item_id'])

In [41]:
ss_train.to_csv('ninja.csv', index=False, sep=';')

In [43]:
ss_test.to_csv('estenio.csv', index=False, sep=';')

# Etapa final.

In [5]:
#ss_train = pd.read_csv('ninja.csv', sep=';')
#ss_train['searchs_int'] = ss_train['searchs_int'].apply(lambda x: literal_eval(x))
#ss_test = pd.read_csv('estenio.csv', sep=';')
#ss_test['searchs_int'] = ss_test['searchs_int'].apply(lambda x: literal_eval(x))

In [44]:
#Esto hay que aplicarlo sí o sí porque viene de la función vectorial como str.
ss_train['encoded_title'] = ss_train['encoded_title'].apply(lambda x: literal_eval(x))

In [45]:
#Diccionario de puntaje de palabras.
ss_train['puntaje'] = None
for i in tqdm(range(len(ss_train))):
    ctitle = ss_train['encoded_title'].iloc[i]
    dicc = {}
    for s in ss_train['searchs_int'].iloc[i]:
        for w in s:
            if w == ctitle[0]:
                dicc.update({w: 36})
            elif w in ctitle:
                dicc.update({w: 12})
            else:
                dicc.update({w: 1})
    #Falla caveats.
    ss_train['puntaje'][i] = dicc

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [46]:
ss_train.to_csv('ninja_con_puntaje.csv', index=False, sep=';')

In [8]:
#ss_train = pd.read_csv('ninja_con_puntaje.csv', sep=';')
#ss_train['puntaje'] = ss_train['puntaje'].apply(lambda x: literal_eval(x))

In [47]:
puntajes = {}
for i in tqdm(list(ss_train['ib'].value_counts().index)):
    compras = ss_train.loc[ss_train['ib'] == i, 'puntaje']
    compras = compras.reset_index()
    dicc = {}
    for j in range(len(compras)):
        for k in compras['puntaje'].iloc[j]:
            if k in dicc:
                dicc[k] = dicc[k] + compras['puntaje'].iloc[j][k]
            else:
                dicc[k] = compras['puntaje'].iloc[j][k]
    puntajes.update({i: dicc})

In [48]:
palabras = {}
for i in tqdm(puntajes):
    for j in puntajes[i]:
        if j in palabras:
            palabras[j].update({i: puntajes[i][j]})
        else:
            palabras[j] = {i: puntajes[i][j]}

In [49]:
%%time
#Tarda dos horas en terminar.
pred = []
k = 10
for i in tqdm(list(ss_train['searchs_int'])):
    temp = join_rankings(i, palabras)
    if len(temp) < k:
        temp.extend([-1] * (k - len(temp)))
    pred.append(temp)
    
pred_pred = []
k = 10
for i in tqdm(list(ss_test['searchs_int'])):
    temp = join_rankings(i, palabras)
    if len(temp) < k:
        temp.extend([-1] * (k - len(temp)))
    pred_pred.append(temp)
    
DIM_SSS = pd.DataFrame(pred, columns=['pred'+str(i) for i in range(10)])
DIM_SSS.to_csv('dim_sss.csv', index=False, sep=';')

DIM_PRED_SSS = pd.DataFrame(pred_pred, columns=['pred'+str(i) for i in range(10)])
DIM_PRED_SSS.to_csv('dim_pred_sss.csv', index=False, sep=';')


Wall time: 2h 39min 36s


In [50]:
a = np.array(DIM_SSS)

In [51]:
b = np.array(DIM_PRED_SSS)

In [52]:
#Valor anteior: 0.8010475284572917.
a[a != -1].size / a.size

0.8226878979966744

In [53]:
#Valor anterior: 0.8012994860789519.
b[b != -1].size / b.size

0.8232614220364828